# **Building an Image Search Engine - Image Encoding**

## Overview


This guided project will help you build an image querying prototype. The project is split into two parts. The first part focuses on building the image encoding system and the second part focuses on building the image query system.

This application has many business use cases such as:

1.  Enabling customers to look for similar apparel, furniture, auto parts etc.
2.  Help eliminate near duplicate images from databases or catalogues.
3.  Enable image to be used as feature embedding for modeling tasks.
4.  Build image based recommendation systems.


## Objectives

After completing this notebook you will be able to:

*   Setup an Image Encoding service that accepts input images and produces embeddings
*   Explore techniques for generating embeddings
*   Generate the embeddings for the dataset and save it on disc


## Setup Runtime

*   we recommend the use of anaconda to manage the runtime.
*   install the dependencies within the Anaconda runtime.


System requirements:

1.  Stable internet access (to download the dataset)
2.  TensorFlow 2.x
3.  Jupyter notebook
4.  2GB of storage


In [3]:
!pip install -U tensorflow

     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     ----------------------------------- ---- 51.2/57.5 kB 2.6 MB/s eta 0:00:01
     -------------------------------------- 57.5/57.5 kB 749.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/65.5 kB ? eta -:--:--
     ---------------------------------------- 65.5/65.5 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.5 MB 2.3 MB/s eta 0:00:01
     --- ------------------------------------ 0.1/1.5 MB 1.7 MB/s eta 0:00:01
     ------ --------------------------------- 0.2/1.5 MB 1.5 MB/s eta 0:00:01
     -------- ------------------------------- 0.3/1.5 MB 1.5 MB/s eta 0:00:01
     ----------- ---------------------------- 0.4/1.5 MB 1.7 MB/s eta 0:00:01
     --------------- ------------------------ 0.6/1.5 MB 2.0 MB/s eta 0:00:01
     ------------------- -------------------- 0.7/1.5 MB 2.0 MB/s eta 0:00:01


[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import tensorflow as tf
import csv
import random
import numpy as np
import pandas as pd
from random import shuffle
import zipfile

import PIL
import PIL.Image as Image

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers
from keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback

from keras.applications.inception_v3 import InceptionV3
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.xception import Xception

import matplotlib.pyplot as plt

## Exercise 1 - Download Image Dataset

For this prototype we will use a clothing dataset of tshirts/apparel created by [Alexey Grigorev](https://github.com/alexeygrigorev). A fork of the dataset can be found [here](https://github.com/CODAIT/clothing-dataset) on IBM CODAIT's GitHub.

*   Click [the link](https://github.com/CODAIT/clothing-dataset) to download the data manually.
*   Save the downloaded dataset to your local file system.

Alternatively you can use the `git clone` command below to download the dataset within the notebook kernel.


In [6]:
# Download the dataset
!git clone https://github.com/CODAIT/clothing-dataset.git

Cloning into 'clothing-dataset'...
Updating files:  18% (1062/5759)
Updating files:  19% (1095/5759)
Updating files:  20% (1152/5759)
Updating files:  21% (1210/5759)
Updating files:  22% (1267/5759)
Updating files:  23% (1325/5759)
Updating files:  24% (1383/5759)
Updating files:  25% (1440/5759)
Updating files:  26% (1498/5759)
Updating files:  27% (1555/5759)
Updating files:  28% (1613/5759)
Updating files:  29% (1671/5759)
Updating files:  30% (1728/5759)
Updating files:  31% (1786/5759)
Updating files:  32% (1843/5759)
Updating files:  33% (1901/5759)
Updating files:  34% (1959/5759)
Updating files:  34% (1984/5759)
Updating files:  35% (2016/5759)
Updating files:  36% (2074/5759)
Updating files:  37% (2131/5759)
Updating files:  38% (2189/5759)
Updating files:  39% (2247/5759)
Updating files:  40% (2304/5759)
Updating files:  41% (2362/5759)
Updating files:  42% (2419/5759)
Updating files:  43% (2477/5759)
Updating files:  44% (2534/5759)
Updating files:  45% (2592/5759)
Updating

## Exercise 2 - Preprocessing the Dataset

We will now build a pandas dataframe that contains image file paths of valid images. We will discard invalid images.
To do this, we will first try to open the images using the `PIL.Image.open()` method and drop the image if the method failed to open the image.


In [7]:
from os import walk 

path = 'clothing-dataset/images/'
filename_list = []

# collect all files
for (dirpath, dirnames, filenames) in walk(path):
    filename_list.extend(filenames)
    break

# validate images
filename_list_verified = []
for index, fname in enumerate(filename_list):
    try:
        im = Image.open(path + fname)
        filename_list_verified.append(fname)
    except Exception as e:
        print('invalid image index:', index)

df = pd.DataFrame(data={'filename': filename_list_verified})
df['full_path_file_name'] = path + df['filename']
df['class'] = "1"
print(df.shape)
df.head()

(5756, 3)


,filename,full_path_file_name,class
0,00003aeb-ace5-43bf-9a0c-dc31a03e9cd2.jpg,clothing-dataset/images/00003aeb-ace5-43bf-9a0...,1
1,00070a23-9faf-4682-8243-59f307880f4b.jpg,clothing-dataset/images/00070a23-9faf-4682-824...,1
2,000d31f6-44b5-4758-8d86-c4fcfb234392.jpg,clothing-dataset/images/000d31f6-44b5-4758-8d8...,1
3,001068a8-86c3-4029-a9d9-1e27015e7e64.jpg,clothing-dataset/images/001068a8-86c3-4029-a9d...,1
4,00143901-a14c-4600-960f-7747b4a3a8cd.jpg,clothing-dataset/images/00143901-a14c-4600-960...,1


Once we have a dataframe of all valid images and their full paths on the disc, we store that as a CSV file for quick access later.


In [8]:
pd.DataFrame.to_csv(df, 'image_dictionary.csv', index=False)

## Exercise 3 - Implement The LSH TF Layer

Local Sensitive Hashing is a popular hashing technique which we will use in our prototype to generate embeddings for the images.
To perform that, we first need to create a custom `Layer` object in Keras/tensorflow.
We can achieve that by doing the following steps:

*   Implement the custom TF layer by subclass the `tf.keras.layers.Layer` class
*   Implement the `HyperPlane Hashing` in the `call()` method of `layer` class

For more details about Local Sensitive Hashing and HyperPlane Hashing please follow this [link](https://web.stanford.edu/class/cs246/slides/03-lsh.pdf?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkQuickLabsIBMGPXX0W3UEN35891832-2022-01-01)


In [9]:
# perform hyper plan hashing
# the output is intended for LSH algorithm
class HyperPlaneHashingLayer(tf.keras.layers.Layer):

    # perform input independent initialization
    # initialize the random_seed
    def __init__(self, n, random_seed=628):
        """
        @param::random_seed: the random seed used to generate the hyperplan
        @param::n: the number of hashing performed
        return none
        """
        #super(tf.keras.layers.Layer, self).__init__()
        super().__init__()
        self.random_seed = random_seed
        self.n = n
        self.hyperplanes = []

    # initialize the hyperplane matrix based on the input size 
    def build(self, input_shape):
        """
        @param::input_shape: the shape of input tensor
        return none
        """
        if len(input_shape) <= 3:
          raise Exception('> input dimension need to greater than 3.')
        tf.random.set_seed(self.random_seed)
        # each column represent a hashing vector
        self.hyperplanes = K.random_uniform((input_shape[-1], self.n),  
                                              minval=-1., 
                                              maxval=1., 
                                              seed=self.random_seed)

    # return the hyperplane hashing result
    def call(self,input):
        """
        @param::input: the input tensor
        return the hyperplane hashed representation of each input data points
        """
        # scaled = K.mean(input, axis=0) - input
        scaled = tf.reshape(input,(-1,input.shape[-2]*input.shape[-3],input.shape[-1]))
        # print(scaled.shape)
        hash_val = tf.matmul(scaled, self.hyperplanes)
        hash_result = (hash_val) > 0
        return K.cast(hash_result, tf.int32)


## Exercise 4 - Build Encoding Network

*   Use the pretrained VGG16 network as the feature extraction network. More detail about [pretrained models](https://keras.io/api/applications/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkQuickLabsIBMGPXX0W3UEN35891832-2022-01-01) in keras.
*   Use the LSH layer built in the previous section to reduce the dimensionality of the embeddings output by the pretrained network


In [10]:
# VGG16
# Preprocessing expected input: 0-255 float32
i = tf.keras.layers.Input([None, None, 3], dtype = tf.float32)
x = tf.keras.applications.vgg16.preprocess_input(i) 
x = VGG16(include_top=False, weights='imagenet', input_shape=(224,224,3))(x)
#x = tf.keras.layers.MaxPool2D()(x)
#x = tf.keras.layers.MaxPool2D()(x)
#x = tf.keras.layers.Flatten()(x)
x = HyperPlaneHashingLayer(500)(x)
model = tf.keras.Model(inputs=[i], outputs=[x])
model.summary()




58889256/58889256 [==============================] - 14s 0us/step


TypeError: Exception encountered when calling layer "hyper_plane_hashing_layer" (type HyperPlaneHashingLayer).

in user code:

    File "C:\Users\Lenovo\AppData\Local\Temp\ipykernel_4244\2544600863.py", line 41, in call  *
        scaled = tf.reshape(input,(-1,input.shape[-2]*input.shape[-3],input.shape[-1]))

    TypeError: unsupported operand type(s) for *: 'NoneType' and 'NoneType'


Call arguments received by layer "hyper_plane_hashing_layer" (type HyperPlaneHashingLayer):
  • input=tf.Tensor(shape=(None, None, None, 512), dtype=float32)